In [ ]:
import datetime
import numpy as np
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd

from preprocessing import *

from pathlib import Path

In [6]:
dataset_filepath = "../../../data/upftfg26/apujols/processed/dataset_FINAL_20251118.csv"

In [8]:
dataset = pd.read_csv(dataset_filepath, sep=";")
dataset.head(5)

,filename,year,month,day,hour,minute,lng,lat,alt,camera,width,height,frames,fps,time,mean_brightness,class
0,M20251004_190700_MasLaRoca_NE,2025.0,7.0,4.0,19.0,7.0,2.19,41.56,570.0,NE,720.0,576.0,62.0,25.0,0.04,70.666667,NaN
1,M20251004_225201_MasLaRoca_NE,2025.0,52.0,4.0,22.0,52.0,2.19,41.56,570.0,NE,720.0,576.0,120.0,25.0,2.40,53.413793,NaN
2,M20251005_043858_MasLaRoca_NE,2025.0,38.0,5.0,4.0,38.0,2.19,41.56,570.0,NE,720.0,576.0,62.0,25.0,0.04,47.500000,NaN
3,M20251005_053814_MasLaRoca_NE,2025.0,38.0,5.0,5.0,38.0,2.19,41.56,570.0,NE,720.0,576.0,62.0,25.0,0.08,40.458333,NaN
4,M20251005_194436_MasLaRoca_NE,2025.0,44.0,5.0,19.0,44.0,2.19,41.56,570.0,NE,720.0,576.0,62.0,25.0,0.04,38.500000,NaN


In [9]:
dataset.describe()

,year,month,day,hour,minute,lng,lat,alt,width,height,frames,fps,time,mean_brightness
count,21149.000000,21149.000000,21149.000000,21149.000000,21149.000000,21149.000000,21149.000000,21149.0,21149.0,21149.0,21149.000000,21149.0,21149.000000,21149.000000
mean,2024.560310,29.596671,16.316989,9.596908,29.596671,2.229588,41.682062,570.0,720.0,576.0,76.497801,25.0,0.609379,72.490921
std,0.496361,17.389150,8.497134,9.003243,17.389150,0.056422,0.173969,0.0,0.0,0.0,59.104910,0.0,2.349304,34.383623
min,2024.000000,0.000000,1.000000,0.000000,0.000000,2.190000,41.560000,570.0,720.0,576.0,61.000000,25.0,0.040000,11.000000
25%,2024.000000,15.000000,10.000000,2.000000,15.000000,2.190000,41.560000,570.0,720.0,576.0,63.000000,25.0,0.080000,48.500000
50%,2025.000000,30.000000,15.000000,4.000000,30.000000,2.190000,41.560000,570.0,720.0,576.0,65.000000,25.0,0.160000,61.500000
75%,2025.000000,45.000000,24.000000,20.000000,45.000000,2.310000,41.930000,570.0,720.0,576.0,71.000000,25.0,0.400000,85.909091
max,2025.000000,59.000000,31.000000,23.000000,59.000000,2.310000,41.930000,570.0,720.0,576.0,2547.000000,25.0,99.480000,255.000000


In [10]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21149 entries, 0 to 21148
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   filename         21149 non-null  object 
 1   year             21149 non-null  float64
 2   month            21149 non-null  float64
 3   day              21149 non-null  float64
 4   hour             21149 non-null  float64
 5   minute           21149 non-null  float64
 6   lng              21149 non-null  float64
 7   lat              21149 non-null  float64
 8   alt              21149 non-null  float64
 9   camera           21149 non-null  object 
 10  width            21149 non-null  float64
 11  height           21149 non-null  float64
 12  frames           21149 non-null  float64
 13  fps              21149 non-null  float64
 14  time             21149 non-null  float64
 15  mean_brightness  21149 non-null  float64
 16  class            13670 non-null  object 
dtypes: float64(1

In [14]:
dataset[['year', 'month', 'day', 'hour', 'minute']] = dataset[['year', 'month', 'day', 'hour', 'minute']].astype(int)
dataset['datetime'] = pd.to_datetime(dataset[['year', 'month', 'day', 'hour', 'minute']], errors='coerce')
dataset.head(5)

,filename,year,month,day,hour,minute,lng,lat,alt,camera,width,height,frames,fps,time,mean_brightness,class,datetime
0,M20251004_190700_MasLaRoca_NE,2025,7,4,19,7,2.19,41.56,570.0,NE,720.0,576.0,62.0,25.0,0.04,70.666667,NaN,2025-07-04 19:07:00
1,M20251004_225201_MasLaRoca_NE,2025,52,4,22,52,2.19,41.56,570.0,NE,720.0,576.0,120.0,25.0,2.40,53.413793,NaN,NaT
2,M20251005_043858_MasLaRoca_NE,2025,38,5,4,38,2.19,41.56,570.0,NE,720.0,576.0,62.0,25.0,0.04,47.500000,NaN,NaT
3,M20251005_053814_MasLaRoca_NE,2025,38,5,5,38,2.19,41.56,570.0,NE,720.0,576.0,62.0,25.0,0.08,40.458333,NaN,NaT
4,M20251005_194436_MasLaRoca_NE,2025,44,5,19,44,2.19,41.56,570.0,NE,720.0,576.0,62.0,25.0,0.04,38.500000,NaN,NaT


In [15]:
dataset.to_csv("../../../data/upftfg26/apujols/processed/dataset_processed.csv")